# Planejamento do projeto

## Resultado final
    - Uma classe em linguagem python em que o usuário digita o símbolo da ação (stock ticker) e recebe uma série 
    de gráficos e análises da empresa selecionada.
    - Quais análises serão entregues?
        . Gráfico interativo com o comportamento das ações no tempo;
        . Relatório com os principais indicadores;
        . Avaliação de investimento, favorável ou desvaforável?
        . Previsão de comportamento?

## Ferramental
    - Utilizar o Jupyter Notebook para a escrita e interpretação do código;
    - Utilizar a biblioteca Selenium ou urllib.request para analisar conteudos de páginas;
    - Utilizar a biblioteca Beautiful Soup que torna possível ler o conteúdo das páginas HTML;
    - Utilizar a biblioteca Pandas para armazenar o conteúdo das páginas em DataFrames;
    - Utilizar a biblioteca Numpy para realização de operações;
    - Utilizar a biblioteca Plotly para desenhar gráficos interativos;
    - Utilizar a biblioetca pyplot da matplotlib para criação de gráficos e dashboards;
    - Utilizar a biblioteca sklearn para a adaptação de um modelo de previsão de preço de ação.
    
## Processo de desenvolvimento
    - Coletar os dados das páginas que contêm informações sobre ações: https://finance.yahoo.com/ 
    - Organizar os dados em DataFrames
    - Tratar os dados, isto é, retirar dados duplicados, tratar dados faltantes, ajustar indices e headers
    - Gerar as análises dos dados